In [2]:
import numpy as np
import pandas as pd
from ratings_graph import RatingsGraph
from baseline_debiaser import debias_ratings_baseline
from tqdm import tqdm

In [24]:
def read_table(data_file, cols_to_cast_float = []):
    raw_data = pd.read_table(data_file)
    
    data = raw_data
    if cols_to_cast_float:
        good_data_rows = np.where(np.all(np.hstack([(raw_data[col_name] != '\\N')[:, None] 
                                                    for col_name in cols_to_cast_float]), axis=1))[0]
        data = raw_data.iloc[good_data_rows].copy()
        
        for col_name in cols_to_cast_float:
            data.loc[:, col_name] = data.loc[:, col_name].astype(float)
        
    return data
    
movies_numeric_cols = ['rtAllCriticsRating', 'rtAllCriticsNumReviews', 'rtAllCriticsNumFresh', 
                    'rtAllCriticsNumRotten', 'rtAllCriticsScore', 'rtTopCriticsRating', 'rtTopCriticsNumReviews', 
                    'rtTopCriticsNumFresh', 'rtTopCriticsNumRotten', 'rtTopCriticsScore', 'rtAudienceRating', 
                    'rtAudienceNumRatings', 'rtAudienceScore']
movies = read_table('data/grouplens/movies.dat', movies_numeric_cols)
user_ratings = read_table('data/grouplens/user_ratedmovies.dat')

movie_index_lookup = {}
counter = 0
true_ratings = np.zeros(movies.shape[0])
for _, movie in tqdm(movies.iterrows()):
    if movie['rtTopCriticsNumReviews' >= 5]:
        movie_index_lookup[movie['id']] = counter
        true_ratings[counter] = movie['rtTopCriticsRating'] / 10
        counter += 1

user_ids = user_ratings['userID'].unique()
user_index_lookup = {user_id: idx for idx, user_id in enumerate(user_ids)}
num_users, num_movies = len(user_index_lookup), len(movie_index_lookup)
ratings_graph = np.zeros((num_users, num_movies))
ratings_matrix = np.zeros((num_users, num_movies))
for _, rating in tqdm(user_ratings.iterrows()):
    if int(rating['movieID']) in movie_index_lookup:
        ratings_graph[user_index_lookup[rating['userID']], movie_index_lookup[int(rating['movieID'])]] = 1
        ratings_matrix[user_index_lookup[rating['userID']], movie_index_lookup[int(rating['movieID'])]] = rating['rating'] / 5
    
# ratings = user_ratings.loc[['userID', 'movieID', 'rating']].set_index('movieID')
# final_movies = ratings.join(movies[['id']], how='inner')[['movieID']].drop_duplicates().set_index('movieID')
# ratings_wide = ratings.join(final_movies, how='inner', on='movieID')\
#                     .pivot(index='userID', columns='movieID', values='rating')

# ratings_graph = np.array(ratings_wide)
# ratings_matrix = np.array(ratings_wide)
# ratings_matrix[np.isnan(ratings_matrix)] = 0
# ratings_graph[np.isnan(ratings_matrix)] = 0
# ratings_graph[np.logical_not(np.isnan(ratings_matrix))] = 1
# true_ratings = final_movies.join(movies, how='inner', on='movieID')

855598it [01:11, 11890.96it/s]


In [27]:
print np.min(np.sum(ratings_graph, axis=0))
print np.any(np.isnan(ratings_matrix))

0.0
False


In [25]:
movie_ratings_graph = RatingsGraph(ratings_graph, ratings_matrix)
biases, true_ratings, errors = debias_ratings_baseline(movie_ratings_graph, 0.2, 100)

True
